In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated, Optional, Dict, Literal
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [2]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model = "gemmi-2.5-flash")

when in future we will store one best mail and tweak based on company name and all 

In [3]:
class email(TypedDict, total=False):
    sender_name : str
    reciver_name : str
    role : str
    email : str
    context : str #we will get this info from our context
    resume : str
    status : Literal["approced", "need_improvement"]
    iteration : int
    max_iteration : int
    email_history : Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]
    #if we have finalised a email we will make only teak each time 
    best_mail : str
    tweak_mail : str

In [4]:
def generate_email(state: email) -> email:
    
    system_prompt = (
        f"You are a senior recruiter with 15+ years of experience hiring for {state['role']} roles. "
        "You understand what recruiters value in cold outreach emails. "
        "Your job is to craft concise, high-impact, professional emails that are achievement-driven, "
        "role-aligned, and context-focused. Avoid fluff. Avoid generic phrases."
    )

    human_prompt = (
        f"I am applying for the role of {state['role']}.\n\n"
        f"Context about me:\n{state['context']}\n\n"
        "Write a short, professional email that:\n"
        "- Highlights measurable achievements\n"
        "- Aligns strongly with the role\n"
        "- Is direct and confident\n"
        "- Is under 150 words\n\n"
        f"Sender Name: {state['sender_name']}\n"
        f"Receiver Name: {state['reciver_name']}\n"
        f"Resume Link: {state['resume']}\n"
    )

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt)
    ]

    response = llm.invoke(messages)
    generated_email = response.content.strip()

    state["email_history"] =  state.get("email_history", []) + [generated_email]
    state["iteration"] = state.get("iteration") + 1
    
    return state